In [1]:
from datasets import load_dataset, load_metric
import datasets
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import random
import torch
from torch.optim import Adam

import transformers
from transformers import AdamW,AutoTokenizer, AutoModelForSequenceClassification,get_linear_schedule_with_warmup, TrainingArguments, Trainer
from torch.optim import Adadelta, Adam, SGD, RMSprop, AdamW
import wandb
#from transformers.trainer_utils import A

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [4]:
task = "mnli"
model_checkpoint = "microsoft/mpnet-base"#"symanto/mpnet-base-snli-mnli"


In [5]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Found cached dataset glue (/home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_5414/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)


In [6]:
#%pip install scikit-learn

In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,"For instance, the Post also has the story about the woman meeting with Clinton just days before his first Inaugural, but adds the detail that she says all the encounters were innocent.",A woman met with Clinton days before his first Inaugural but it wasn't thought to be dirty.,entailment,132307
1,"The sky had ripped again, and this time the entire dome shook with the shock.",The dome was shaking as the sky ripped once more.,entailment,360074
2,it's really scary that you have to be that scared about your kids but i mean you do better safe than sorry,It's bad that you have to be that scared for your kids.,entailment,293276
3,He had no time to ask.,There was no time to question it.,entailment,70657
4,Further Reform Is Needed to Address Long-Standing Problems,There is a need for more reform.,entailment,275800
5,"It told me that Mrs. Inglethorp had been writing the word 'possessed' that afternoon, and, having the fragment of paper found in the grate fresh in my mind, the possibility of a will, (a document almost certain to contain that word), occurred to me at once.",I thought the will may contain the word possessed.,entailment,263292
6,"Fortunately, many of these lawyers are willing and able to help in other ways.",Many lawyers are available to help in a variety of ways.,entailment,252147
7,But the components of that growth tell the real story.,The real story is hard to come by without the components of that growth.,neutral,352362
8,The others fled.,The others fled from where they were.,entailment,329686
9,"By the way, Hastings, there is something I want you to do for me.",I have a favor that I want to ask of you.,entailment,7729


In [8]:
#print(metric)

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
print(metric.compute(predictions=fake_preds, references=fake_labels,))


{'accuracy': 0.625}


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.", truncation=True)

{'input_ids': [0, 7596, 1014, 2027, 2032, 6255, 1003, 2, 2, 2002, 2027, 6255, 3636, 2011, 2013, 1016, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [12]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


In [13]:
# def preprocess_function(examples):
#     if sentence2_key is None:
#         return tokenizer(examples[sentence1_key], truncation=True)
#     return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], padding=True, truncation=True)
    return tokenizer(
        examples[sentence1_key],
        examples[sentence2_key],
        padding=True,
        truncation=True
    )

In [14]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 17162, 2139, 6953, 8305, 25061, 2042, 2052, 3941, 9650, 1015, 4035, 2002, 10509, 1016, 2, 2, 4035, 2002, 10509, 2028, 2058, 2195, 6953, 8305, 25061, 2151, 1016, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2021, 2117, 2080, 2000, 2165, 2002, 1049, 3988, 2016, 2016, 2119, 2508, 7914, 2021, 4562, 2072, 2004, 2000, 2283, 2508, 2069, 2069, 2031, 5634, 2004, 9135, 2000, 2000, 6691, 2140, 2000, 13984, 5634, 2004, 2659, 2004, 9135, 1041, 3128, 2017, 6424, 1041, 2063, 1041, 3317, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2002, 1041, 2313, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2, 2, 2021, 4562, 2000, 2481, 2004, 2000, 2210, 2508, 2069, 2000, 2115, 9135, 1016, 2], [0, 2032, 2001, 2260, 2197, 2101, 4291, 2045, 2119, 8132, 3375, 2139, 1016, 2, 2, 1041, 2270, 2001, 2030, 2140, 2101, 15393, 2119, 4453, 2011, 14273, 11722, 1016, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

Preprocessing

In [15]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e918085800bdaf35.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-19eaff48a7946f26.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fbc8360796e1193d.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d767475232a1637b.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-61b7eb49a8b6e726.arrow


In [16]:

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.o

In [17]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
batch_size = 4
optimizer_name = "AdamW" #AdamW, SGD, Adadelta
args = TrainingArguments(
    f"{model_name}-finetuned-{optimizer_name}-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=6.7e-5,#6.7e-5(81.7),#1.8e-5,#2.4e-5 , #3e-5 AdamW
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=0.20,
    weight_decay=1.09,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
    report_to="wandb",
)
#run num 1      19      18        18        17      16      15      14      13      12      11      10      9       8       7       6       5       4       3       2       1
#accuracy score 82.4    82.8      83.55     83.1    82.4    83.1    81.7   82.17    81.7
#lr             6.7e-5  6.7e-5    6.7e-5    6.7e-5  6.7e-5  6.7e-5  6.7e-5 6.7e-5   6.7e-5  6.7e-4  6.7e-4  6.7e-4
#weight_decay   1.07    1.05      1.08    1.1     1.14      1.1      1.0    0.8     0.8     0.01    0.5     0.5      0.5     0.5     0.5     0.5     0.5     0.5     0.5     0.5
#                       real      redo

In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [19]:
#estimated_training_size =args.size_of_dataset if args.size_of_dataset is not None else len(encoded_dataset["train"])
estimated_training_size = len(encoded_dataset["train"])
num_epoch = args.num_train_epochs*100 if args.num_train_epochs is not None else 1

print(num_epoch, estimated_training_size,args.num_train_epochs)
total_step = estimated_training_size * num_epoch // args.per_device_train_batch_size
if total_step <= 0:
        t_total = estimated_training_size * num_epoch // args.actual_train_batch_size
else:
    t_total = total_step

if args.warmup_steps <= 0:  # set the warmup steps to 0.1 * total step if the given warmup step is -1.
    args.warmup_steps = int(t_total * 0.1)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
#optimizer = SGD(optimizer_grouped_parameters, lr=args.learning_rate, momentum=0.0, weight_decay=args.weight_decay) #pas reussi lr=3e-6 3e-5 3e-4 3e-3 3e-2
#optimizer=Adadelta(optimizer_grouped_parameters, lr=args.learning_rate, weight_decay=args.weight_decay,) #lr=3e-6 
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
)

20.0 392702 0.2


In [20]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers=(optimizer, scheduler),
    #optimizers=(transformers.AdamW, transformers.get_scheduler("linear", num_warmup_steps=0, num_training_steps=args.num_train_epochs*len(encoded_dataset["train"])/args.per_device_train_batch_size)),
    #token=token
)

/home/nico/Documents/IAS2/IAssistant/AI-base/mpnet-base-finetuned-AdamW-mnli is already a clone of https://huggingface.co/NicolasLe/mpnet-base-finetuned-AdamW-mnli. Make sure you pull the latest changes with `repo.git_pull()`.


In [21]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise. If idx, hypothesis, premise are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
/home/nico/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 392702
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 19636
  Number of trainable parameters = 109488771
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] =

  0%|          | 0/19636 [00:00<?, ?it/s]

You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.1, 'learning_rate': 1.7061283110348304e-07, 'epoch': 0.01}
{'loss': 1.0974, 'learning_rate': 3.412256622069661e-07, 'epoch': 0.01}
{'loss': 1.0978, 'learning_rate': 5.118384933104492e-07, 'epoch': 0.02}
{'loss': 1.097, 'learning_rate': 6.824513244139321e-07, 'epoch': 0.02}
{'loss': 1.0967, 'learning_rate': 8.530641555174153e-07, 'epoch': 0.03}
{'loss': 1.0904, 'learning_rate': 1.0236769866208984e-06, 'epoch': 0.03}
{'loss': 1.0635, 'learning_rate': 1.1942898177243815e-06, 'epoch': 0.04}
{'loss': 1.0092, 'learning_rate': 1.3649026488278643e-06, 'epoch': 0.04}
{'loss': 0.9365, 'learning_rate': 1.5355154799313475e-06, 'epoch': 0.05}
{'loss': 0.8622, 'learning_rate': 1.7061283110348306e-06, 'epoch': 0.05}
{'loss': 0.7956, 'learning_rate': 1.8767411421383134e-06, 'epoch': 0.06}
{'loss': 0.7268, 'learning_rate': 2.047353973241797e-06, 'epoch': 0.06}
{'loss': 0.7159, 'learning_rate': 2.2179668043452795e-06, 'epoch': 0.07}
{'loss': 0.6757, 'learning_rate': 2.388579635448763e-06, 'ep

The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: idx, hypothesis, premise. If idx, hypothesis, premise are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 4


...

Saving model checkpoint to mpnet-base-finetuned-AdamW-mnli/checkpoint-19636
Configuration saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/config.json


{'eval_loss': 0.5741181969642639, 'eval_accuracy': 0.8290371879775853, 'eval_runtime': 124.3193, 'eval_samples_per_second': 78.95, 'eval_steps_per_second': 19.739, 'epoch': 0.2}


Model weights saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/pytorch_model.bin
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/special_tokens_map.json
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from mpnet-base-finetuned-AdamW-mnli/checkpoint-19636 (score: 0.8290371879775853).


{'train_runtime': 4861.8945, 'train_samples_per_second': 16.154, 'train_steps_per_second': 4.039, 'train_loss': 0.7586076246499772, 'epoch': 0.2}


TrainOutput(global_step=19636, training_loss=0.7586076246499772, metrics={'train_runtime': 4861.8945, 'train_samples_per_second': 16.154, 'train_steps_per_second': 4.039, 'train_loss': 0.7586076246499772, 'epoch': 0.2})

In [22]:
#print(metric.compute(predictions=fake_preds, references=fake_labels,))
trainer.push_to_hub(model_name=f"{model_name}-finetuned-{optimizer_name}-{task}")


Saving model checkpoint to mpnet-base-finetuned-AdamW-mnli
Configuration saved in mpnet-base-finetuned-AdamW-mnli/config.json
Model weights saved in mpnet-base-finetuned-AdamW-mnli/pytorch_model.bin
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/NicolasLe/mpnet-base-finetuned-AdamW-mnli
   ffed91e..7b79114  main -> main



In [ ]:
# #fitetuningpretrained pytorch from scratch
# del model
# del trainer
# del args
# #del batch
# torch.cuda.empty_cache()